In [ ]:
import requests
import bs4
from dotenv import load_dotenv
import os
import psycopg

load_dotenv()

In [ ]:
dbconn = os.getenv("DBCONN")
dbconn

In [3]:
conn = psycopg.connect(dbconn)
cur = conn.cursor()

In [4]:
# cur.execute('''
# CREATE TABLE IF NOT EXISTS financial_times_scaped(
#     tag VARCHAR(255),
#     link VARCHAR(255),
#     heading VARCHAR(255),
#     teaser VARCHAR(255),
#     date TIMESTAMP,
#     FOREIGN KEY (date) REFERENCES bitcoin_api_data(date)
# );
# ''')

In [5]:
def results_for_date(date):
  response = requests.get(f"https://www.ft.com/search?q=bitcoin&dateFrom={date}&dateTo={date}&sort=relevance")
  scraped_date = []
  print(response.status_code)
  if response.status_code == 200:
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    limit = 5
    search_results = soup.select("div.o-teaser__content", limit=limit)
    for result in search_results:
      scraped = [date]
      try:
        scraped.append(result.select_one("a.o-teaser__tag").get_text())
      except:
        scraped.append("")
      try:
        scraped.append(result.select_one("a.js-teaser-heading-link").get_text())
      except:
        scraped.append("")
      try:
        scraped.append(result.select_one("a.js-teaser-heading-link")["href"])
      except:
        scraped.append("")
      try:
        scraped.append(result.select_one("p.o-teaser__standfirst > a").get_text())
      except:
        scraped.append("")
      scraped_date.append(tuple(scraped))
  return scraped_date

In [ ]:
cur.execute('''
SELECT date FROM bitcoin_api_data WHERE date < '2024-08-02'::date ORDER BY date DESC;
''')

In [7]:
date_times = cur.fetchall()
dates = [date[0].strftime('%Y-%m-%d') for date in date_times]

In [ ]:
print(len(dates), "remaining")

In [ ]:
# only does ∼10 days before 406
for date in dates:
  scraped = results_for_date(date)
  print(scraped)
  if len(scraped) == 0:
    break
  for item in scraped:
    print("adding item", item)
    cur.execute(
      '''
        INSERT INTO financial_times_scaped(date, tag, heading, link, teaser)
        VALUES (%s, %s, %s, %s, %s);
      ''', 
      item
    )
print("done")

In [10]:
# cur.execute(
#   '''TRUNCATE TABLE financial_times_scaped;'''
# )

In [11]:
conn.commit()
cur.close()
conn.close()